In [1]:
dbutils.widgets.text("storage_account", "")
account_name = getArgument("storage_account")
dbutils.widgets.text("storage_container", "")
container_name = getArgument("storage_container")

if not account_name:
  raise "account_name must be set"
  
if not container_name:
  raise "contain_name must be set"

In [2]:
# Remove existing mounts to make notebook idempotent
try:
  dbutils.fs.unmount("/mnt/landing") 
  dbutils.fs.unmount("/mnt/output")
except Exception as e:
  print(e)

/mnt/landing has been unmounted.
/mnt/output has been unmounted.

In [3]:
dbutils.widgets.text("storage_account", "")
account_name = getArgument("storage_account")
dbutils.widgets.text("storage_container", "")
container_name = getArgument("storage_container")

app_id = dbutils.secrets.get("terraform", "datalake_sp_applicationid")
client_secret = dbutils.secrets.get("terraform", "datalake_sp_app_client_secret")
tenant = dbutils.secrets.get("terraform", "datalake_sp_tenant")

configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": app_id,
       "fs.azure.account.oauth2.client.secret": client_secret,
       "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant}/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

# Mount landing
dbutils.fs.mount(
  source = f"abfss://{container_name}@{account_name}.dfs.core.windows.net/landing",
  mount_point = "/mnt/landing",
  extra_configs = configs)

#Mount output
dbutils.fs.mount(
  source = f"abfss://{container_name}@{account_name}.dfs.core.windows.net/output",
  mount_point = "/mnt/output",
  extra_configs = configs)


In [4]:
import os.path
import IPython
import time
from pyspark.sql import SQLContext

# Check landing
dbutils.fs.put("/mnt/landing/created", f"DBFS Mount created at {time.gmtime()}", True)
print(dbutils.fs.ls("/mnt/landing"))

# Check output
dbutils.fs.put("/mnt/output/created", f"DBFS Mount created at {time.gmtime()}", True)
print(dbutils.fs.ls("/mnt/output"))


Wrote 142 bytes.
[FileInfo(path='dbfs:/mnt/landing/created', name='created', size=142)]
Wrote 142 bytes.
[FileInfo(path='dbfs:/mnt/output/created', name='created', size=142)]